# Prescient Tutorial

## Getting Started
This is a tutorial to demonstration the basic functionality of Prescient. Please follow the installation instructions in the [README](https://github.com/grid-parity-exchange/Prescient/blob/master/README.md) before proceeding. This tutorial will assume we are using the CBC MIP solver, however, we will point out where one could use a different solver (CPLEX, Gurobi, Xpress).

## RTS-GMLC
We will use the RTS-GMLC test system as a demonstration. Prescient comes included with a translator for the RTS-GMLC system data, which is publically available [here](https://github.com/GridMod/RTS-GMLC). To find out more about the RTS-GMLC system, or if you use the RTS-GMLC system in published research, please see or cite the [RTS-GMLC paper](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8753693&isnumber=4374138&tag=1).

## IMPORTANT NOTE
In the near future, the dev-team will allow more-direct reading of data in the "RTS-GMLC" format directly into the simulator. In the past, we have created one-off scripts for each data set to put then in the format required by the populator.

### Downloading the RTS-GMLC data

In [ ]:
# first, we'll use the built-in function to download the RTS-GMLC system to Prescicent/downloads/rts_gmlc
import prescient.downloaders.rts_gmlc as rts_downloader

# the download function has the path Prescient/downloads/rts_gmlc hard-coded.
# All it does is a 'git clone' of the RTS-GMLC repo
rts_downloader.download()

In [ ]:
# we should be able to see the RTS-GMLC data now
import os
rts_gmlc_dir = rts_downloader.rts_download_path+os.sep+'RTS-GMLC'
print(rts_gmlc_dir)
os.listdir(rts_gmlc_dir)

### Converting RTS-GMLC data into the format for the "populator"

In [ ]:
# first thing we'll do is to create a *.dat file template for the "static" data, e.g.,
# branches, buses, generators, to Prescicent/downloads/rts_gmlc/templates/rts_with_network_template_hotstart.dat
from prescient.downloaders.rts_gmlc_prescient.rtsgmlc_to_dat import write_template
write_template(rts_gmlc_dir=rts_gmlc_dir,
        file_name=rts_downloader.rts_download_path+os.sep+'templates'+os.sep+'rts_with_network_template_hotstart.dat')

In [ ]:
# next, we'll convert the included time-series data into input for the populator
# (this step can take a while because we set up an entire year's worth of data)
from prescient.downloaders.rts_gmlc_prescient.process_RTS_GMLC_data import create_timeseries
create_timeseries(rts_downloader.rts_download_path)

In [ ]:
# Lastly, Prescient comes with some pre-made scripts and templates to help get up-and-running with RTS-GMLC.
# This function just puts those in rts_downloader.rts_download_path from 
# Prescient/prescient/downloaders/rts_gmlc_prescient/runners
rts_downloader.copy_templates()
os.listdir(rts_downloader.rts_download_path)

NOTE: the above steps are completely automated in the `__main__` function of Prescient/prescient/downloaders/rts_gmlc.py

### Running the populator
Below we'll show how the populator is set-up by the scripts above an subsequently run.

In [ ]:
# we'll work in the directory we've set up now for
# running the populator and simulator

# If prescient is properly installed, this could be
# a directory anywhere on your system
os.chdir(rts_downloader.rts_download_path)
os.getcwd()

In [ ]:
# helper for displaying *.txt files in jupyter
def print_file(file_n):
    '''prints file contents to the screen'''
    with open(file_n, 'r') as f:
        for l in f:
            print(l.strip())

Generally, one would call `runner.py populate_with_network_deterministic.txt` to set-up the data for the simulator. We'll give a brief overview below as to how that is orchestrated.

In [ ]:
print_file('populate_with_network_deterministic.txt')

First, notice the `command/exec` line, which tells `runner.py` which command to execute. These `*.txt` files could be replaced with bash scripts, or run from the command line directly. In this case,

`populator.py --start-date 2020-07-10 --end-date 2020-07-16 --source-file sources_with_network.txt --output-directory deterministic_with_network_scenarios --scenario-creator-options-file deterministic_scenario_creator_with_network.txt 
--traceback`

would give the same result. The use of the `*.txt` files enables saving these complex commands in a cross-platform compatable manner.

The `--start-date` and `--end-date` specify the date range for which we'll generate simulator input. The `--ouput-directory` gives the path (relative in this case) where the simulator input (the output of this script) should go. The `--sources-file` and `--scenario-creator-options-file` point to other `*.txt` files.

#### --scenario-creator-options-file

In [ ]:
print_file('deterministic_scenario_creator_with_network.txt')

This file points the `scenario_creator` to the templates created/copied above, which store the "static" prescient data, e.g., `--sceneario-template-file` points to the bus/branch/generator data. The `--tree-template-file` is depreciated at this point, pending re-introdcution of stochastic unit commitment capabilities.


In [ ]:
# This prints out the files entire contents, just to look at.
# See if you can find the set "NondispatchableGenerators"
print_file('templates/rts_with_network_template_hotstart.dat')

#### --sources-file

In [ ]:
print_file('sources_with_network.txt')

This file connects each "Source" (e.g., `122_HYDRO_1`) in the file `templates/rts_with_network_template_hotstart.dat` to the `*.csv` files generated above for both load and renewable generation. Other things controlled here are whether a renewable resource is dispatchable at all.

In [ ]:
# You could also run 'runner.py populate_with_network_deterministic.txt' from the command line
import prescient.scripts.runner as runner
runner.run('populate_with_network_deterministic.txt')

This creates the "input deck" for July 10, 2020 -- July 16, 2020 for the simulator in the ouput directory `determinstic_with_network_scenarios`.

In [ ]:
sorted(os.listdir('deterministic_with_network_scenarios'+os.sep+'pyspdir_twostage'))

Inside each of these directories are the `*.dat` files specifying the simulation for each day.

In [ ]:
sorted(os.listdir('deterministic_with_network_scenarios'+os.sep+'pyspdir_twostage'+os.sep+'2020-07-10'))

`Scenario_actuals.dat` contains the "actuals" for the day, which is used for the SCED problems, and `Scenario_forecast.dat` contains the "forecasts" for the day. The other `*.dat` files are hold-overs from stochastic mode.

`scenarios.csv` has forecast and actuals data for every uncertain generator in an easy-to-process format.

### Running the simulator
Below we show how to set-up and run the simulator.

Below is the contents of the included `simulate_with_network_deterministic.txt`:

In [ ]:
print_file('simulate_with_network_deterministic.txt')

Description of the options included are as follows:
 - `--data-directory`: Where the source data is (same as outputs for the populator).
 - `--simulate-out-of-sample`: This option directs the simulator to use different forecasts from actuals. Without it, the simulation is run with forecasts equal to actuals
 - `--run-sced-with-persistent-forecast-errors`: This option directs the simulator to use forecasts (adjusted by the current forecast error) for sced look-ahead periods, instead of using the actuals for sced look-ahead periods.
 - `--output-directory`: Where to write the output data.
 - `--run-deterministic-ruc`: Directs the simualtor to run a deterministic (as opposed to stochastic) unit commitment problem. Required for now as stochastic unit commitment is currently deprecated.
 - `--start-date`: Day to start the simulation on. Must be in the data-directory.
 - `--num-days`: Number of days to simulate, including the start date. All days must be included in the data-directory.
 - `--sced-horizon`: Number of look-ahead periods (in hours) for the real-time economic dispatch problem.
 - `--traceback`: If enabled, the simulator will print a trace if it failed.
 - `--random-seed`: Unused currently.
 - `--output-sced-initial-conditions`: Prints the initial conditions for the economic dispatch problem to the screen.
 - `--output-sced-demands`: Prints the demands for the economic dispatch problem to the screen.
 - `--output-sced-solutions`: Prints the solution for the economic dispatch problem to the screen.
 - `--output-ruc-initial-conditions`: Prints the initial conditions for the unit commitment problem to the screen.
 - `--output-ruc-solutions`: Prints the commitment solution for the unit commitment problem to the screen.
 - `--output-ruc-dispatches`: Prints the dispatch solution for the unit commitment problem to the screen.
 - `--output-solver-logs`: Prints the logs from the optimization solver (CBC, CPLEX, Gurobi, Xpress) to the screen.
 - `--ruc-mipgap`: Optimality gap to use for the unit commitment problem. Default is 1% used here -- can often be tighted for commerical solvers.
 - `--symbolic-solver-labels`: If set, `symbolic_solver_labels` is used when writing optimization models from Pyomo to the solver. Only useful for low-level debugging.
 - `--reserve-factor`: If set, overwrites any basic reserve factor included in the test data.
 - `--deterministic-ruc-solver`: The optimization solver ('cbc', 'cplex', 'gurobi', 'xpress') used for the unit commitment problem.
 - `--sced-solver`: The optimization solver ('cbc', 'cplex', 'gurobi', 'xpress') used for the economic dispatch problem.

Other options not included in this file, which may be useful:
 - `--compute-market-settlements`: (True/False) If enabled, solves a day-ahead pricing problem (in addition to the real-time pricing problem) and computes generator revenue based on day-ahead and real-time prices.
 - `--day-ahead-pricing`: ('LMP', 'ELMP', 'aCHP') Specifies the type of day-ahead price to use. Default is 'aCHP'.
 - `--price-threashold`: The maximum value for the energy price ($/MWh). Useful for when market settlements are computed to avoid very large LMP values when load shedding occurs.
 - `--reserve-price-threashold`: The maximum value for the reserve price (\$/MW). Useful for when market settlements are computed to avoid very large LMP values when reserve shortfall occurs.
 - `--deterministic-ruc-solver-options`: Options to pass into the unit commitment solver (specific to the solver used) for every unit commitment solve.
 - `--sced-solver-options`: Options to pass into the economic dispatch solve (specific to the solver used) for every economic dispatch solve.
 - `--plugin`: Path to a Python module to modify Prescient behavior.

In [ ]:
# You could also run 'runner.py simulate_with_network_deterministic.txt' from the command line
# This runs a week of RTS-GMLC, which with the open-source cbc solver will take several (~12) minutes
import prescient.scripts.runner as runner
runner.run('simulate_with_network_deterministic.txt')

### Analyzing results
Summary and detailed `*.csv` files are written to the specified output directory (in this case, `deterministic_with_network_simulation_output`).

In [ ]:
sorted(os.listdir('deterministic_with_network_simulation_output/'))

Below we give a breif description of the contents of each file.
- `bus_detail.csv`: Detailed results (demand, LMP, etc.) by bus.
- `daily_summary.csv`: Summary results by day. Demand, renewables data, costs, load shedding/over generation, etc.
- `hourly_gen_summary.csv`: Gives total thermal headroom and data on reserves (shortfall, price) by hour.
- `hourly_summary.csv`: Summary results by hour. Similar to `daily_summary.csv`.
- `line_detail.csv`: Detailed results (flow in MW) by bus.
- `overall_simulation_output.csv`: Summary results for the entire simulation run. Similar to `daily_summary.csv`.
- `plots`: Directory containing stackgraphs for every day of the simulation.
- `renewables_detail.csv`: Detailed results (output, curtailment) by renewable generator.
- `runtimes.csv`: Runtimes for each economic dispatch problem.
- `thermal_detail.csv`: Detailed results (dispatch, commitment, costs) per thermal generator.

Generally, the first think to look at, as a sanity check is the stackgraphs:

In [ ]:
dates = [f'2020-07-1{i}' for i in range(0,7)]
from IPython.display import Image
for date in dates:
    display(Image('deterministic_with_network_simulation_output'+os.sep+'plots'+os.sep+'stackgraph_'+date+'.png',
                   width=500))

Due to the non-deterministic nature of most MIP solvers, your results may be slightly different than mine. For my simulation, two things stand out:
1. The load-shedding at the end of the day (hour 23) on July 12th.
2. The renewables curtailed the evening of July 15th into the morning of July 16th.

For this tutorial, let's hypothesize about the cause of (2). Often renewables are curtailed either because of a binding transmission constraint, or because some or all of the thermal generators are operating at minimum power. Let's investigate the first possibility.

#### Examining Loaded Transmission Lines

In [ ]:
import pandas as pd
# load in the output data for the lines
line_flows = pd.read_csv('deterministic_with_network_simulation_output'+os.sep+'line_detail.csv', index_col=[0,1,2,3])

# load in the source data for the lines
line_attributes = pd.read_csv('RTS-GMLC'+os.sep+'RTS_Data'+os.sep+'SourceData'+os.sep+'branch.csv', index_col=0)

# get the line limits
line_limits = line_attributes['Cont Rating']

# get a series of flows
line_flows = line_flows['Flow']

In [ ]:
line_flows

In [ ]:
# rename the line_limits to match the
# index of line_flows
line_limits.index.name = "Line"
line_limits

In [ ]:
lines_relative_flow = line_flows/line_limits

In [ ]:
lines_near_limits_time = lines_relative_flow[ (lines_relative_flow > 0.99) | (lines_relative_flow < -0.99) ]

In [ ]:
lines_near_limits_time

As we can see, near the end of the day on July 15th and the beginning of the day July 16th, several transmission constraints are binding, which correspond exactly to the periods of renewables curtailment in the stackgraphs above.